# Exchange Rates ETL Pipeline

In [55]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import connect
from datetime import datetime

## Extract data from ExchangeRate.host

In [124]:
base_currency = 'USD'
url = f"https://api.frankfurter.app/latest?from={base_currency}"

response = requests.get(url)
data = response.json()

In [126]:
#USe the APIs date 

timestamp = datetime.strptime(data.get('date', datetime.now().strftime("%Y-%m-%d")), "%Y-%m-%d")
rates = data.get('rates', {})

In [128]:
#Create DataFrame

df_rates = pd.DataFrame(rates.items(), columns=["target_currency", "rate"])
df_rates["base_currency"] = base_currency
df_rates["timestamp"] = timestamp

print("\nSample Extracted data : ")
print(df_rates.head())


Sample Extracted data : 
  target_currency     rate base_currency  timestamp
0             AUD  1.57090           USD 2025-04-17
1             BGN  1.72170           USD 2025-04-17
2             BRL  5.88120           USD 2025-04-17
3             CAD  1.38850           USD 2025-04-17
4             CHF  0.81787           USD 2025-04-17


## Load into MYSQL

In [140]:
# Connect to SQL
try:
    conn = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = 'Sinha@9000',
        database = 'exchange_rates_db'
    )

    cursor = conn.cursor()
    print("Connected to SQL")
except Error as e:
    print("Error:", e)

Connected to SQL


In [144]:
#Create table if not exists.

cursor.execute("""
    create table if not exists exchange_rates(
	id int auto_increment primary key,
    base_currency varchar(10),
    target_currency varchar(10),
    rate float,
    timestamp datetime,
    unique (base_currency, target_currency, timestamp)
)
""")

In [146]:
#Inserting Values in the database

for _, row in df_rates.iterrows():
    cursor.execute("""
        insert ignore into exchange_rates(base_currency, target_currency, rate, timestamp)
        values(%s, %s, %s, %s)
    """,(
        row['base_currency'],
        row['target_currency'],
        row['rate'],
        row['timestamp']
    ))

conn.commit()
print("\nData Inserted into MySQL table : exchange_rates")


Data Inserted into MySQL table : exchange_rates


## Query & Analysis

In [153]:
# Query 1 : Highest Exchange Rate

df_highest = pd.read_sql("""
    Select target_currency, rate
    From exchange_rates
    Where base_currency = 'USD'
    Order By rate DESC
    Limit 10
""",conn)
print("\n Highest Exchange Rate: ")
print(df_highest)


 Highest Exchange Rate: 
  target_currency       rate
0             IDR  16868.000
1             KRW   1416.480
2             HUF    358.800
3             JPY    142.590
4             ISK    127.730
5             INR     85.400
6             PHP     56.679
7             TRY     38.081
8             THB     33.275
9             CZK     22.015


C:\Users\admin\AppData\Local\Temp\ipykernel_7488\1142949486.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_highest = pd.read_sql("""


In [159]:
# Query 2 : Currencies with rate > 1

df_gt1 = pd.read_sql("""
    Select target_currency, rate
    From exchange_rates
    Where base_currency = 'USD' AND rate > 1
    Order By rate DESC
    
""",conn)
print("\n Exchange Rate > 1: ")
print(df_gt1.head())


 Exchange Rate > 1: 
  target_currency      rate
0             IDR  16868.00
1             KRW   1416.48
2             HUF    358.80
3             JPY    142.59
4             ISK    127.73


C:\Users\admin\AppData\Local\Temp\ipykernel_7488\2780212385.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_gt1 = pd.read_sql("""


In [161]:
 # Query 3 : Compare USD to EUR, GBP, INR, JPY

df_compare = pd.read_sql("""
    Select target_currency, rate
    From exchange_rates
    Where base_currency = 'USD' AND target_currency IN ('EUR', 'GBP', 'INR', 'JPY')
""",conn)

print("\n USD Compared to Major Currencies: ")
print(df_compare)


 USD Compared to Major Currencies: 
  target_currency       rate
0             EUR    0.88028
1             GBP    0.75592
2             INR   85.40000
3             JPY  142.59000


C:\Users\admin\AppData\Local\Temp\ipykernel_7488\3360551123.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_compare = pd.read_sql("""


In [169]:
# Save Query Results

output_path = "usd_vs_major_currencies.csv"
df_compare.to_csv(output_path, index = False)
print(f'\n Comparison saved to : {output_path}')

cursor.close()
conn.close()


 Comparison saved to : usd_vs_major_currencies.csv
